In [1]:
#initial imports
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# imports for closing pop ups
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [3]:
#ran into pop ups when automated browser clicks "next". 
# working around problem by opening and closing a new browser for each page of the website.


# create a list of urls to visit
urls = []

# add url for universities that start with 0-9 to list of urls
url = ("https://www.4icu.org/reviews/index0001.htm")
urls.append(url)
# print(urls)


# add urls for universities that start with A-Z to list of urls
#convert int to str found (https://stackoverflow.com/questions/52499991/how-to-append-a-integer-iteration-into-a-string-in-python)
# for url in urls:
for i in range(2, 28):       
    url = 'https://www.4icu.org/reviews/index'+ str(i) +'.htm'
    urls.append(url)
    
print(urls)

['https://www.4icu.org/reviews/index0001.htm', 'https://www.4icu.org/reviews/index2.htm', 'https://www.4icu.org/reviews/index3.htm', 'https://www.4icu.org/reviews/index4.htm', 'https://www.4icu.org/reviews/index5.htm', 'https://www.4icu.org/reviews/index6.htm', 'https://www.4icu.org/reviews/index7.htm', 'https://www.4icu.org/reviews/index8.htm', 'https://www.4icu.org/reviews/index9.htm', 'https://www.4icu.org/reviews/index10.htm', 'https://www.4icu.org/reviews/index11.htm', 'https://www.4icu.org/reviews/index12.htm', 'https://www.4icu.org/reviews/index13.htm', 'https://www.4icu.org/reviews/index14.htm', 'https://www.4icu.org/reviews/index15.htm', 'https://www.4icu.org/reviews/index16.htm', 'https://www.4icu.org/reviews/index17.htm', 'https://www.4icu.org/reviews/index18.htm', 'https://www.4icu.org/reviews/index19.htm', 'https://www.4icu.org/reviews/index20.htm', 'https://www.4icu.org/reviews/index21.htm', 'https://www.4icu.org/reviews/index22.htm', 'https://www.4icu.org/reviews/index23

In [4]:
#create loop to open new browser, visit each url in the urls list, extract the data, and close the browser

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}

#create an empty list for the table data
row_data = []

for i in urls:
    
    # set up browser
    browser = Browser('chrome', **executable_path, headless=False) 
               
    # Visit UniRank website
    url = i
    browser.visit(url)
    browser.is_element_present_by_css('div.list_text', wait_time=1)

    # parse the html
    html = browser.html
    html_soup = soup(html, 'html.parser')

    #isolate the table
    universities_table = html_soup.find('table', class_='table table-hover text-left')

    #loop through rows, collect data, append data to list, close browser
    for i in range(len(urls)):
        for row in universities_table.find_all('tr'):
            col = row.find_all('td')
            col = [ele.text.strip()for ele in col]
            row_data.append(col)
            
    # close browser          
    browser.quit()
        
# print(row_data)

In [5]:
university_df = pd.DataFrame(row_data, columns=["University Name", "Country"])
university_df

,University Name,Country
0,None,None
1,3iL École d'ingénieurs,fr
2,None,None
3,3iL École d'ingénieurs,fr
4,None,None
...,...,...
375079,ZNRF University of Management Sciences,bd
375080,Zunyi Medical University,cn
375081,Zunyi Normal College,cn
375082,Zürcher Fachhochschule,ch


In [11]:
university_df.to_csv('C:/Users/sarah/OneDrive/Desktop/Global_GDP_Trends/Resources/webscraped_university_names.csv', index=True)